# 유튜브 랭킹 데이터 수집하기

In [ ]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
# time.sleep(2)

In [ ]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [ ]:
channel = channel_list[0]
category = channel.select_one('p.category').get_text().strip(' \n[]')
category

'음악/댄스/가수'

In [ ]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [ ]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

In [ ]:
channels = []
for channel in channel_list:
  category = channel.select_one('p.category').get_text().strip(' \n[]')
  subscriber = channel.select_one('.subscriber_cnt').text
  name = channel.select_one('.subject a').text.strip()
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.video_cnt').text[:-1]
  channels.append([category, subscriber, name, view, video])


In [ ]:
df = pd.DataFrame(channels, columns=['카테고리', '구독자수','이름', '조회수', '비디오수'])
df

,카테고리,구독자수,이름,조회수,비디오수
0,음악/댄스/가수,6400만,BLACKPINK,190억0381만,371
1,음악/댄스/가수,6030만,HYBE LABELS,187억1305만,654
2,음악/댄스/가수,5640만,BANGTANTV,121억9496만,"1,579"
3,음악/댄스/가수,2850만,SMTOWN,218억7092만,"3,729"
4,키즈/어린이,2650만,Boram Tube Vlog [보람튜브 브이로그],110억5288만,223
...,...,...,...,...,...
95,음식/요리/레시피,408만,까니짱 [ G-NI ],10억3471만,482
96,키즈/어린이,408만,로미유 스토리[Romiyu Story],10억6557만,433
97,음악/댄스/가수,386만,NCT DREAM,3억7422만,228
98,BJ/인물/연예인,381만,허팝Heopop,31억3892만,"1,847"


##만과 억을 숫자로 바꿔주는 함수

In [ ]:
def convert_unit(s):
  s = s.replace('억', '').replace('개','').replace(',','')
 # s = ''.join(s.split('억'))
  s = s.replace('만', '0000')
  return s

In [ ]:
convert_unit('10억5000만')

'1050000000'

## 두 번째 페이지

In [ ]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [ ]:
results = []
for channel in channel_list:
    category = channel.select_one('p.category').get_text().strip(' \n[]')
    name = channel.select_one('.subject a').text.strip()
    subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
    view = convert_unit(channel.select_one('.view_cnt').text)
    video = convert_unit(channel.select_one('.video_cnt').text)
    results.append([category,name,subscriber,view,video])

In [ ]:
df = pd.DataFrame(results, columns=['카테고리', '구독자수','채널명', '조회수', '비디오수'])
df

,카테고리,구독자수,채널명,조회수,비디오수
0,애완/반려동물,크림히어로즈,3800000,1374770000,940
1,BJ/인물/연예인,워크맨-Workman,3780000,679370000,141
2,BJ/인물/연예인,waveya 2011,3730000,1346280000,702
3,키즈/어린이,Lime Tube[라임튜브],3730000,2595420000,1723
4,음악/댄스/가수,WINNER,3730000,1171260000,428
...,...,...,...,...,...
95,음식/요리/레시피,푸드킹덤 Food Kingdom,2090000,744530000,797
96,음악/댄스/가수,4Minute 포미닛(Official YouTube Channel),2090000,1236000000,82
97,미분류,김종국 GYM JONG KOOK,2080000,54080000,10
98,음식/요리/레시피,ZOEY ASMR 조이,2070000,475380000,606


## page 1에서 10까지 크롤링하기

In [ ]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [ ]:
df = pd.DataFrame(results, columns=['카테고리', '구독자수','채널명', '조회수', '비디오수'])
df

,카테고리,구독자수,채널명,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,64000000,19003810000,371
1,음악/댄스/가수,HYBE LABELS,60300000,18713050000,654
2,음악/댄스/가수,BANGTANTV,56400000,12194960000,1579
3,음악/댄스/가수,SMTOWN,28500000,21870920000,3729
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],26500000,11052880000,223
...,...,...,...,...,...
995,게임,미소,510000,210770000,3084
996,미분류,밉지않은 관종언니,510000,67740000,170
997,미분류,복지마블TV [Welfare Marble],510000,49960000,240
998,음식/요리/레시피,Muggo,510000,183720000,1388


In [ ]:
df.to_csv('유튜브_순위.csv', index=None)

In [ ]:
driver.close()

In [ ]:
import random
i = 10
 
while i < 19:
    print(i, end=' ')
    i += 2

10 12 14 16 18 